In [ ]:
#import packages
import pandas as pd
import numpy as np
import IPython.core.display as ipd
import os
import sys
import time
import matplotlib.pyplot as plt
from collections import Counter
import pickle

#set configurations
pd.set_option('max_columns', 1000)
pd.set_option('max_rows', 1000)

%load_ext autoreload
%autoreload 2

from Biometrics import RecognitionSystem

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

%matplotlib inline

# Completo

In [ ]:
inic = time.time()

param_grid = [
    {'method':['OCSVM', 'M2005', 'Statistic'],
     'base' : ['easy', 'strong', 'logical', 'DSL', 'greyc'],
    'impostor_rate' : [0.3, 0.5, 0.7],
    'model_size' : [40],
    'rate_external_impostor' : [0.5],
    'sampling': ['Random', 'GenFirst', 'ImpFirst'],
    'normalize': [False],
    'adaptive' : [False, 'GrowingWindow', 'SlidingWindow']}
]

System = RecognitionSystem(verbose=True, save_results=False)
System.fit(user_column='subject', param_grid=param_grid)
fim = time.time() - inic

temp = System.summary()
temp.to_csv("resultados/OCSVM_growing.csv", index=False)